In [1]:
pip install pyspark scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.2/434.2 MB 6.5 MB/s eta 0:00:00m eta 0:00:010:00:02
  Preparing metadata (setup.py) ... one
  DEPRECATION: Building 'pyspark' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyspark'. Discussion can be found at https://github.com/pypa/pip/issues/6334
done
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813860 sha256=8b7ff4554ddbc35bfe2af78c1e2b4bf9df7a11d8d7ac8ff2c9008c48cd79d77b
  Stored in directory: /home/sakhawat/.cache/pip/wheels/31/9f/68/f89fb34ccd886909be7d0e390eaaf97f21efdf540c0ee8dbcd
Successfully built pyspark
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pyspark]━━━ 1/2 [pyspark]
Note: you may need

In [2]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
features = pd.DataFrame(iris.data, columns=iris.feature_names)   # requirement (1)
target   = pd.DataFrame(iris.target, columns=['label'])


In [3]:
iris_df = pd.concat([features, target], axis=1)   # column-wise concat
iris_df.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Iris_Classification_PySpark") \
    .master("local[*]") \
    .getOrCreate()

spark_df = spark.createDataFrame(iris_df)
spark_df.printSchema()
spark_df.show(5)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/05 23:01:26 WARN Utils: Your hostname, sakhawat, resolves to a loopback address: 127.0.1.1; using 10.147.114.173 instead (on interface wlp3s0)
25/11/05 23:01:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/05 23:01:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- label: long (nullable = true)



[Stage 0:>                                                          (0 + 1) / 1]

+-----------------+----------------+-----------------+----------------+-----+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|label|
+-----------------+----------------+-----------------+----------------+-----+
|              5.1|             3.5|              1.4|             0.2|    0|
|              4.9|             3.0|              1.4|             0.2|    0|
|              4.7|             3.2|              1.3|             0.2|    0|
|              4.6|             3.1|              1.5|             0.2|    0|
|              5.0|             3.6|              1.4|             0.2|    0|
+-----------------+----------------+-----------------+----------------+-----+
only showing top 5 rows


In [5]:
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)
print("Train count:", train_df.count(), "Test count:", test_df.count())


NameError: name 'final_df' is not defined

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100)
model = lr.fit(train_df)   # fit distributed model


In [ ]:
result = (model, test_df)
# result[0] is the trained model, result[1] is the Spark DataFrame test set


In [ ]:
predictions = model.transform(test_df)
# show sample
predictions.select("features","label","prediction","probability").show(5)

# compute accuracy with MulticlassClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test accuracy:", accuracy)


In [6]:
# ============================================================
# Project 1 — Iris Classification using PySpark ML LogisticRegression
# ============================================================

# Step 1. Import all required libraries
import pandas as pd
from sklearn.datasets import load_iris
from pyspark.sql import SparkSession
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Step 2. Load the Iris dataset from scikit-learn
iris = load_iris()

# Step 3. Convert numpy arrays to pandas DataFrames
features = pd.DataFrame(iris.data, columns=iris.feature_names)
target = pd.DataFrame(iris.target, columns=["label"])

# Step 4. Combine features and target using concat
iris_df = pd.concat([features, target], axis=1)
print("✅ Combined pandas DataFrame created successfully.")

# Step 5. Initialize SparkSession
spark = SparkSession.builder \
    .appName("Iris_Classification_PySpark") \
    .master("local[*]") \
    .getOrCreate()
print("✅ Spark session started.")

# Step 6. Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(iris_df)
print("✅ Converted to Spark DataFrame.")
spark_df.show(5)

# Step 7. Use RFormula to map feature columns -> features and target -> label
feature_cols = iris.feature_names
formula = RFormula(formula="label ~ " + " + ".join(feature_cols))
rf_model = formula.fit(spark_df)
final_df = rf_model.transform(spark_df)
print("✅ RFormula applied successfully.")
final_df.select("features", "label").show(5)

# Step 8. Split the data into training (80%) and test (20%)
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)
print(f"✅ Data split: {train_df.count()} training rows, {test_df.count()} test rows")

# Step 9. Initialize and train (fit) LogisticRegression model
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100)
model = lr.fit(train_df)
print("✅ Model training completed.")

# Step 10. Return the tuple (model, test_df)
result = (model, test_df)
print("✅ Tuple (model, test_df) created successfully.")

# Step 11. Test model predictions
predictions = model.transform(test_df)
print("✅ Predictions generated.")
predictions.select("features", "label", "prediction", "probability").show(10, truncate=False)

# Step 12. Evaluate model performance
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"🎯 Model accuracy on test set: {accuracy:.4f}")

# Optional: show more evaluation metrics
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
print(f"🔹 F1 Score: {evaluator_f1.evaluate(predictions):.4f}")

# Done
print("\n✅ Project completed successfully! Logistic Regression model is trained and evaluated.")


✅ Combined pandas DataFrame created successfully.
✅ Spark session started.
✅ Converted to Spark DataFrame.
+-----------------+----------------+-----------------+----------------+-----+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|label|
+-----------------+----------------+-----------------+----------------+-----+
|              5.1|             3.5|              1.4|             0.2|    0|
|              4.9|             3.0|              1.4|             0.2|    0|
|              4.7|             3.2|              1.3|             0.2|    0|
|              4.6|             3.1|              1.5|             0.2|    0|
|              5.0|             3.6|              1.4|             0.2|    0|
+-----------------+----------------+-----------------+----------------+-----+
only showing top 5 rows


IllegalArgumentException: Could not parse formula: label ~ sepal length (cm) + sepal width (cm) + petal length (cm) + petal width (cm)